In [27]:
import os
import json
import csv
import datetime
from collections import defaultdict
import cv2
import time
import supervision as sv
from ultralytics import YOLO
import numpy as np

# --------------------------------------------------
# AI SECURITY SYSTEM — PEOPLE DETECTED ON ZONE OVERLAP
# - Correction: Ajout d'une période de grâce pour gérer l'occlusion
# --------------------------------------------------

# --- CONFIG ---
VIDEO_PATH = "Easy_test_output2.mp4"
OUTPUT_DIR = "analytics"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CLOSE_DIST = 80
CLOSE_TIME = 3.0
RUN_SPEED = 120
FALL_HEIGHT_RATIO = 0.4
CROWD_THRESH = 8

# NOUVELLE CONSTANTE: Période de grâce (en secondes) avant de considérer un ID comme "parti"
OCCLUSION_GRACE_PERIOD = 5

# Default green zone
ZONE_POLY = np.array([[100, 200], [400, 200], [400, 500], [100, 500]], dtype=np.int32)

# --- ROI INPUT ---
print("Enter ROI (space-separated x,y pairs). Example: 100,200 400,200 400,500 100,500")
while True:
    user_input = input("Enter ROI points: ").strip()
    if user_input == "":
        print("Using default ROI.")
        break
    try:
        pts = [list(map(int, token.split(','))) for token in user_input.split()]
        if len(pts) < 3:
            print("Need at least 3 points. Try again.")
            continue
        ZONE_POLY = np.array(pts, dtype=np.int32)
        print("ROI accepted.")
        break
    except Exception:
        print("Invalid input. Example: 100,200 400,200 400,500 100,500")

# --- LOGGING ---
log_file = os.path.join(OUTPUT_DIR, "events.csv")
with open(log_file, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["timestamp", "event", "details"])

def log_event(event, details=""):
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([ts, event, details])
    print(f"LOG → {event} | {details}")

# --- MODEL & TRACKER ---
model = YOLO("yolov8n.pt")
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5)

# --- STATE ---
# MODIFIÉ: Structure pour inclure last_seen_time
clients_present = {}        # Format: { tid: {"entry_time": t, "last_seen_time": t} }
zone_ever_entered = set()   # For total entry count
total_zone_entries = 0
path_history = defaultdict(list)
height_history = {}
close_pairs = {}
robbery_alerted = set()
finished_durations = []
final_stats = {}

Enter ROI (space-separated x,y pairs). Example: 100,200 400,200 400,500 100,500


Enter ROI points:  850,350 10,550 10,1400 2700,1400 2700,700


ROI accepted.


In [28]:


# --- VIDEO SETUP ---
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise FileNotFoundError(f"Cannot open video: {VIDEO_PATH}")

ret, first_frame = cap.read()
if not ret:
    cap.release()
    raise RuntimeError("Cannot read first frame")

h, w = first_frame.shape[:2]  # Frame height and width for mask

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS) or 20
frame_size = (w, h)
out_full = cv2.VideoWriter(os.path.join(OUTPUT_DIR, "output_full.mp4"), fourcc, fps, frame_size)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Clip helpers
clip_writer = None
clip_start_time = None
clip_path = None

def start_clip():
    global clip_writer, clip_start_time, clip_path
    if clip_writer: return
    clip_start_time = time.time()
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    clip_path = os.path.join(OUTPUT_DIR, f"alert_{ts}.mp4")
    clip_writer = cv2.VideoWriter(clip_path, fourcc, fps, frame_size)
    log_event("CLIP_START", os.path.basename(clip_path))

def stop_clip():
    global clip_writer, clip_path
    if clip_writer:
        clip_writer.release()
        log_event("CLIP_SAVED", os.path.basename(clip_path))
        clip_writer = None

# --- MAIN LOOP ---
print("Processing... (output saved to analytics folder)")
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    t_now = time.time()
    frame_idx += 1

    # --- DETECTION & TRACKING (ONLY IF BOX OVERLAPS ZONE) ---
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Keep only person class
    try:
        mask_person = np.array(detections.class_id) == 0
        detections = detections[mask_person]
    except Exception:
        pass

    # Create zone mask
    zone_mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(zone_mask, [ZONE_POLY], 255)

    # Filter detections where box overlaps zone
    if len(detections) > 0:
        inside_mask = []
        for box in detections.xyxy:
            x1, y1, x2, y2 = map(int, box)
            # Clip to frame bounds
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
            if x2 > x1 and y2 > y1:
                roi = zone_mask[y1:y2, x1:x2]
                inside_mask.append(np.any(roi > 0))
            else:
                inside_mask.append(False)
        inside_mask = np.array(inside_mask)
        detections = detections[inside_mask]

    # Track only people overlapping the zone
    detections = tracker.update_with_detections(detections)
    current_ids = set(detections.tracker_id) if hasattr(detections, "tracker_id") and detections.tracker_id is not None else set()

    # --- CENTROIDS (only for overlapping detections) ---
    centroids = {}
    if len(detections) > 0:
        boxes = np.array(detections.xyxy)
        for i, tid in enumerate(detections.tracker_id):
            x1, y1, x2, y2 = boxes[i]
            centroids[tid] = np.array([(x1 + x2) / 2, (y1 + y2) / 2])

    # --- (DÉBUT) BLOC DE LOGIQUE CORRIGÉ POUR L'OCCLUSION ---
    
    # --- 2. Gérer les NOUVEAUX et Mettre à jour les ANCIENS ---
    for tid in current_ids:
        if tid not in clients_present:
            # C'est une NOUVELLE personne DANS la zone
            clients_present[tid] = {
                "entry_time": t_now,
                "last_seen_time": t_now
            }
          
            total_zone_entries += 1
            log_event("ZONE_ENTER", f"ID {tid}")
        else:
            # C'est une personne qu'on connaît, elle est toujours là
            # On met à jour son 'last_seen_time'
            clients_present[tid]["last_seen_time"] = t_now

    # --- 3. Gérer les DISPARUS (avec période de grâce) ---
    for tid in list(clients_present.keys()):
        # Vérifier si l'ID a disparu ET si sa période de grâce est écoulée
        if tid not in current_ids:
            time_since_last_seen = t_now - clients_present[tid]["last_seen_time"]
            
            if time_since_last_seen > OCCLUSION_GRACE_PERIOD:
                # La période de grâce est terminée, on le supprime VRAIMENT
                entry_time = clients_present[tid]["entry_time"]
                
                # IMPORTANT : Le "vrai" temps de sortie est 'last_seen_time'
                last_seen_time = clients_present[tid]["last_seen_time"]
                dwell = last_seen_time - entry_time # Durée de présence réelle
                
                finished_durations.append(dwell)
                log_event("ZONE_EXIT", f"ID {tid} ({dwell:.1f}s)")
                
                final_stats[tid] = {
                    "entry": datetime.datetime.fromtimestamp(entry_time).isoformat(),
                    # Utiliser last_seen_time comme "exit"
                    "exit": datetime.datetime.fromtimestamp(last_seen_time).isoformat(), 
                    "duration_sec": dwell,
                    "path": path_history.get(tid, [])
                }
                del clients_present[tid]
        
        # Note: Si l'ID est 'not in current_ids' mais que time_since_last_seen <= GRACE_PERIOD,
        # on ne fait rien ! On lui laisse une chance de réapparaître.
        
    # --- (FIN) BLOC DE LOGIQUE CORRIGÉ ---


    # --- ROBBERY ---
    ids_list = sorted(current_ids)
    for i, id1 in enumerate(ids_list):
        for id2 in ids_list[i+1:]:
            if id1 not in centroids or id2 not in centroids: continue
            dist = np.linalg.norm(centroids[id1] - centroids[id2])
            key = tuple(sorted([int(id1), int(id2)]))
            if dist < CLOSE_DIST:
                if key not in close_pairs:
                    close_pairs[key] = {"start": t_now, "centroids": []}
                close_pairs[key]["centroids"].append((centroids[id1].copy(), centroids[id2].copy()))
            else:
                close_pairs.pop(key, None)

    for key, data in list(close_pairs.items()):
        if t_now - data["start"] >= CLOSE_TIME and key not in robbery_alerted and len(data["centroids"]) >= 2:
            c1p, c2p = data["centroids"][-2]
            c1c, c2c = data["centroids"][-1]
            s1 = np.linalg.norm(c1c - c1p) * fps
            s2 = np.linalg.norm(c2c - c2p) * fps
            if s1 > RUN_SPEED or s2 > RUN_SPEED:
                id1, id2 = key
                log_event("ROBBERY", f"IDs {id1}-{id2}")
                robbery_alerted.add(key)
                start_clip()

    # --- FALL ---
    fall_ids = set()
    for i, tid in enumerate(getattr(detections, "tracker_id", [])):
        x1, y1, x2, y2 = np.array(detections.xyxy)[i]
        h_val = y2 - y1
        hist = height_history.setdefault(tid, [])
        hist.append(h_val)
        if len(hist) > 10: hist.pop(0)
        if len(hist) == 10 and h_val < np.mean(hist) * FALL_HEIGHT_RATIO:
            fall_ids.add(tid)
            log_event("FALL", f"ID {tid}")
            start_clip()

    # --- CROWD (in zone) ---
    if len(current_ids) > CROWD_THRESH:
        log_event("CROWD", f"{len(current_ids)} people in zone")

    # --- TRAJECTORY ---
    for tid, c in centroids.items():
        path_history[tid].append(c.tolist())
        if len(path_history[tid]) > 50:
            path_history[tid].pop(0)

    # --- ANNOTATIONS ---
    labels = []
    # Note: On utilise 'getattr' car 'detections' peut être vide
    for tid in getattr(detections, "tracker_id", []):
        if tid in clients_present:
            elapsed = t_now - clients_present[tid]["entry_time"]
            labels.append(f"ID:{tid} ({elapsed:.0f}s)")
        else:
            labels.append(f"ID:{tid}")

    annotated = box_annotator.annotate(frame.copy(), detections)
    annotated = label_annotator.annotate(annotated, detections, labels=labels)

    # Robbery line
    for key in robbery_alerted:
        id1, id2 = key
        if id1 in centroids and id2 in centroids:
            cv2.line(annotated, tuple(centroids[id1].astype(int)), tuple(centroids[id2].astype(int)), (0,0,255), 4)
            cv2.putText(annotated, "ROBBERY!", (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

    # Fall boxes
    if fall_ids:
        mask = np.isin(detections.tracker_id, list(fall_ids))
        try:
            fall_dets = detections[mask]
            annotated = sv.BoxAnnotator(color=sv.Color.red(), thickness=4).annotate(annotated, fall_dets)
        except Exception:
            for i, tid in enumerate(detections.tracker_id):
                if tid in fall_ids:
                    x1, y1, x2, y2 = map(int, detections.xyxy[i])
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 0, 255), 3)

    # Zone + stats
    cv2.polylines(annotated, [ZONE_POLY], True, (0,255,0), 3)
    cv2.putText(annotated, f"People in Zone: {len(clients_present)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(annotated, f"Total Entries: {total_zone_entries}", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    if finished_durations:
        cv2.putText(annotated, f"Avg Time: {np.mean(finished_durations):.1f}s", (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    if len(current_ids) > CROWD_THRESH:
        cv2.putText(annotated, f"CROWD! {len(current_ids)}", (50, 170), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

    # Trajectories
    for pts in path_history.values():
        for a, b in zip(pts[:-1], pts[1:]):
            cv2.line(annotated, tuple(map(int,a)), tuple(map(int,b)), (255,255,0), 2)

    # --- CLIP & OUTPUT ---
    if clip_writer:
        clip_writer.write(annotated)
        if time.time() - clip_start_time > 5:
            stop_clip()
    out_full.write(annotated)

# --- CLEANUP ---
out_full.release()
cap.release()
if clip_writer: clip_writer.release()
cv2.destroyAllWindows()


# --- SAVE STATS ---
# Logguer tous les clients qui étaient encore présents à la fin
for tid, data in clients_present.items():
    final_stats[tid] = {
        "entry": datetime.datetime.fromtimestamp(data["entry_time"]).isoformat(),
        "exit": None, # N'est pas sorti
        "duration_sec": None, # La durée n'est pas finalisée
        "path": path_history.get(tid, [])
    }

  # --- SAVE RESULTAT.TXT ---
txt_path = os.path.join(OUTPUT_DIR, "resultat.txt")
with open(txt_path, "w", encoding="utf-8") as f:
    f.write("=== TEMPS PASSE PAR PERSONNE DANS CHAQUE SOUS-ZONE ===\n\n")

    for tid, stats in final_stats.items():  # <- virgule ajoutée
        f.write(f"Personne ID: {tid}\n")
        f.write(f" - Entrée: {stats['entry']}\n")
        f.write(f" - Sortie: {stats['exit']}\n")
        f.write(f" - Durée totale: {stats['duration_sec']} secondes\n")
        

print(f"Résultats enregistrés dans: {txt_path}")
  

print(f"\nDONE! → ./{OUTPUT_DIR}")
print("output_-full.mp4")
print("events.csv") 

Processing... (output saved to analytics folder)
LOG → ZONE_ENTER | ID 1
LOG → ZONE_ENTER | ID 5
LOG → ZONE_ENTER | ID 8
LOG → ZONE_EXIT | ID 1 (22.1s)
LOG → ZONE_ENTER | ID 10
LOG → ZONE_ENTER | ID 11
LOG → ZONE_ENTER | ID 12
LOG → ZONE_ENTER | ID 13
LOG → ZONE_ENTER | ID 14
LOG → ZONE_ENTER | ID 15
LOG → ZONE_EXIT | ID 11 (0.3s)
LOG → ROBBERY | IDs 12-13
LOG → CLIP_START | alert_20251116_125609.mp4
LOG → ZONE_EXIT | ID 10 (1.7s)
LOG → ZONE_EXIT | ID 12 (0.1s)
LOG → ZONE_EXIT | ID 13 (0.2s)
LOG → ZONE_EXIT | ID 14 (0.0s)
LOG → ZONE_EXIT | ID 15 (0.6s)
LOG → CLIP_SAVED | alert_20251116_125609.mp4
LOG → ZONE_ENTER | ID 16
LOG → ZONE_ENTER | ID 19
LOG → ZONE_ENTER | ID 20
LOG → ZONE_EXIT | ID 16 (8.1s)
LOG → ZONE_ENTER | ID 24
LOG → ROBBERY | IDs 5-20
LOG → CLIP_START | alert_20251116_125640.mp4
LOG → ZONE_EXIT | ID 19 (1.4s)
LOG → ZONE_EXIT | ID 20 (0.1s)
LOG → ZONE_EXIT | ID 5 (58.8s)
LOG → CLIP_SAVED | alert_20251116_125640.mp4
LOG → ZONE_ENTER | ID 25
LOG → ZONE_ENTER | ID 26
LOG → Z